In [35]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import time, random
from fake_useragent import UserAgent

fake_user = UserAgent().random
print(fake_user)
sleep = random.uniform(4,6)
url = "https://www.twse.com.tw/zh/page/trading/exchange/STOCK_DAY_AVG.html"
options = webdriver.ChromeOptions()
options.add_argument("User-Agent="+fake_user)
options.add_argument("Connection=close")
driver = webdriver.Chrome( chrome_options=options,
                          executable_path="./chromedriver")

driver.get(url)
time.sleep(sleep)

Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; SLCC1; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729; .NET CLR 1.1.4322)


In [36]:
time.sleep(sleep)
s = driver.find_element_by_css_selector("#d1 > select:nth-child(2)")
Select(s).select_by_value("6")
time.sleep(sleep)

In [37]:
stuck_input = driver.find_element_by_css_selector("#main-form > div > div > form > input")
stuck_input.send_keys("0050")
time.sleep(sleep)

In [38]:
search = driver.find_element_by_css_selector("#main-form > div > div > form > a.button.search")
search.click()

In [ ]:
df = pd.read_html(driver.page_source)

In [40]:
from bs4 import BeautifulSoup as bs
time.sleep(sleep)
soup = bs(driver.page_source, "html.parser")

In [47]:
td0_c = []
td1_c = []
for tr in soup.find("table").find("tbody").find_all("tr"):
    tds = tr.find_all("td")
#     print("日期: "+tds[0].text, "收盤價: "+tds[1].text)
    td0_c.append(tds[0].text)
    td1_c.append(tds[1].text)

In [60]:
import pandas as pd

# method 1
stock_d_c = {"日期":td0_c,"收盤價":td1_c}
df_c1 = pd.DataFrame(stock_d_c)

# method 2
df_c2 = pd.DataFrame()
df_c2["日期"] = td0_c
df_c2["收盤價"] = td1_c

In [68]:
# method 3
td_r = {}
for i,tr in enumerate(soup.find("table").find("tbody").find_all("tr")):
    tds = tr.find_all("td")
#     print("日期: "+tds[0].text, "收盤價: "+tds[1].text)
    td_r[i]=[tds[0].text, tds[1].text]

stock_c = ["日期","收盤價"]

In [69]:
df_r = pd.DataFrame.from_dict(td_r, orient='index',
                                   columns=stock_c)


In [44]:
driver.quit()

109-06-01 84.90
109-06-02 85.30
109-06-03 86.80
109-06-04 87.60
109-06-05 88.35
109-06-08 89.50
109-06-09 89.60
109-06-10 90.45
109-06-11 89.05
109-06-12 88.25
109-06-15 87.00
109-06-16 88.90
109-06-17 89.00
109-06-18 88.90
109-06-19 88.85
109-06-22 89.05
109-06-23 89.45
109-06-24 90.00
109-06-29 89.10
109-06-30 89.90


In [136]:
# import pymysql
# import pandas as pd

# ip = "192.168.56.102"
# user = "harold"
# pw = "123456"
# db = "testdb1"
# sql = """
#     select * from YT;
# """
# sql_insert = "insert into YT values({}, '{}', {});"

# def pymysqlcon(ip, user, pw, db ,sql):
#     db = pymysql.connect(ip, user, pw, db)
#     cursor = db.cursor(pymysql.cursors.DictCursor)#拿到dict
# #     cursor.execute("insert into YT values(0, 109-06-01, 84.90);")

# #     save data to YT (id ,date, cp)   (int, date, float)
#     for i in range(df_r.shape[0]-1):
#         date_str_l = df_r.iloc[i,0].split('/')
#         date_str_l[0] = str(int(date_str_l[0]) + 1911)
#         date_str = date_str_l[0]+"-"+date_str_l[1]+"-"+date_str_l[2]
#         cursor.execute(sql_insert.format(i,date_str,df_r.iloc[i,1]))
    
#     db.commit()#commit是把查詢語句提交到資料庫內
#     accounts = cursor.fetchall()#拿到dict
#     db.close()
#     return db, cursor ,accounts

# db, cursor ,accounts = pymysqlcon(ip, user, pw, db ,sql)
# user = pd.DataFrame(accounts)
# user

""


In [140]:
from pymongo import MongoClient

#無認證的連線方式
# client = MongoClient(host='192.168.56.105', port=27017)
# #or
client = MongoClient('mongodb://192.168.56.105:27017/')

# #有認證的連線方式
# client = MongoClient('example.com',
#                       username='user',
#                       password='password',
#                       authMechanism='MONGODB-CR')
# or
# uri = "mongodb://user:[email protected]/?authSource=the_database&authMechanism=MONGODB-CR"
# client = MongoClient(uri)
#創建mongoDB
db = client["testdb123"]
#創建mongo_collection
col = db["YT"]

records = df_r.to_dict('records') # 參數 record 代表把列轉成個別物件
col.insert_many(records)